In [52]:
# Dependencies
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer

In [53]:
# Importing data
df = pd.read_table("Lucas_unique_directors_naturalperson_gender.csv", header= None, sep= None, engine="python")

# Reshaping data frame
data_prev = df.rename(columns=df.iloc[0])
data_inter = data_prev.drop(labels=0,axis="index")
data = data_inter.drop("naturalperson", axis=1)
data # Desired data frame shape

,isin,v1,companynamelatinalphabet,bvdidnumber,name,cname,dmfullname,dmuciuniquecontactidentifie,dmjobtitleinenglish,dmjobtitle,...,dmcorp,gender,dmcorrespondingbvdidwhenapp,dmasanycategory,dmcurrentorprevious,dmbirthplace,dmhasasignatoryright,dmhasapowerofattorney,dmnoofcosinwhichacurrent,compensationsalaryeur
1,NL0012015705,2225,TAKEAWAY.COM N.V.,NL08142836,JUST EAT TAKEAWA,JUST EAT TAKEAWAY COM NV,GRIBHOLD B.V.,C000459947,Director,Bestuurder,...,Previous,NaN,NL06089183,No,Previous,NaN,No,No,0,NaN
2,NL0000383800,1837,SMIT INTERNATIONALE NV,NL24004888,SMIT INTL.CERTS. DEAD - 05/05/10,SMIT INTL.,Boskalis Holding B.V.,C001024491,Directeur,Directeur,...,Current,NaN,NL23056607,Yes,Current,NaN,No,No,3,NaN
3,NL0000289783,12368,ROBECO GLOBAL STARS EQUITIES FUND N.V.,NL24041906,ROBECO SUST.GLB. STARS EQTIES.FD.,ROBECO GLB.STARS EQ.FD.,Robeco Fund Management B.V.,C001055291,Director,Bestuurder,...,Previous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NL0000350361,3289,EXACT HOLDING NV,NL27234422,EXACT HOLDING DEAD - 31/03/15,EXACT HOLDING NV,Exact Management B.V.,C001133291,Directeur,Directeur,...,Current,NaN,NL27228442,No,Current,NaN,No,No,NaN,NaN
5,NL0009508720,13696,LBI INTERNATIONAL N.V.,NL30277334,LBI INTERNATIONAL DEAD - 07/03/13,LBI INTERNATIONAL NV,Fint Management B.V.,C001200105,Director,Bestuurder,...,Previous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123508,CH0496451508,359,SOFTWAREONE HOLDING AG,CHCHE384378612,SOFTWAREONE HOLDING,SOFTWAREONE HOLDING AG,Daniel Von Stockar,P649256001,Chairman Supervisory Board,Chairman Supervisory Board,...,Current,NaN,NaN,No,Current,NaN,No,No,34,NaN
123509,CY0009731015,8461,XXI CENTURY INVESTMENTS PUBLIC LIMITED,CYC132218,XXI CENTURY INVS.PUBLIC DEAD - 03/12/13,XXI CEN.INVS.PUBLIC LTD.,Ms Stella Aristotelous,P649258636,Director,Director,...,Current,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123510,GRS228003000,4116,IMERYS INDUSTRIAL MINERALS GREECE S.A.,GR094000952,S&B INDUSTRIAL MRLS. DEAD - 12/07/13,S&B INDUSTRIAL MRLS.SA,Anastasia Amvrosiadou,P649259308,Chief Executive Officer,Chief Executive Officer,...,Current,NaN,NaN,No,Current,NaN,No,No,1,NaN
123511,CY0007400613,8843,CHRIS JOANNOU PUBLIC LTD,CYC12013,CHRIS IOANNOU,CHRIS IOANNOU PCL.,Mr Georgios Evripidou,P649264673,Director,Director,...,Current,M,NaN,No,Current,NaN,No,No,1,NaN


### Task 1: Creating dummy variable 'natural person'

__Logic:__ Using custom binary technique based on string attributes. 
   1. I am checking if column dmfullname has special addressing techniques (e.g. Sir, Dr., Madam ...);
   2. I am checking if column dmfullname has any non-natural person attribute (e.g. numbers, punctuation);
   3. I am checking if column dmfullname has any non-natural person addressing techniques for main regions mapped in the data(e.g. company entity registrations like GmbH or Ltd);

In [54]:
# First filter: Title identifier

# Part 1: function special_person_address_identifier identifies which observations 
# have one of the titles options in dmfullname and creates intermediary columns named
# after the title list option with 1 for positive, e.g. there is a title in dmfullnane,
# or 0 if there is no.

def special_person_address_identifier(address):
    for i in address:
        data[i] = np.where(data["dmfullname"].str.contains(i), 1, 0)
    
titles = ["Sir", "Madam", "Ms", "Mr", "Mrs","Miss", "Dr", "Professor"]
special_person_address_identifier(titles)

# Part 2: I have merge all intermediary title columns, e.g. Sir, into one column called 'TitleCheck'.
# I have then used the same process as Part 1 to investigate which observations had a positive value of 1 and which not.
# This would indicate then which have a positive 'FilterTitle' status. The ones that have are likely natural person while
# the ones with a negative status, a.k.a 0 value, not.

data["TitleCheck"] = data[titles].apply(lambda row: "+" .join(row.values.astype(str)), axis=1)

data["FilterTitle"] = np.where(data["TitleCheck"].str.contains("1"), 1, 0)
data = data.drop(["Sir", "Madam", "Ms", "Mr", "Mrs","Miss", "Dr", "Professor","TitleCheck"], axis=1)
data["FilterTitle"].value_counts()

1    115982
0      7530
Name: FilterTitle, dtype: int64

In [55]:
# Second filter: Non-natural person identifier (Numbers, Signals and other)

def non_natural_identifier(symbols):
    for i in symbols:
        data[i] = np.where(data["dmfullname"].str.contains(i), 1, 0)

symbols_list = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
non_natural_identifier(symbols_list)

data["SymbolsCheck"] = data[symbols_list].apply(lambda row: "+" .join(row.values.astype(str)), axis=1)

data["FilterSymbols"] = np.where(data["SymbolsCheck"].str.contains("1"), 1, 0)
data = data.drop(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "SymbolsCheck"], axis=1)
data["FilterSymbols"].value_counts() # Result shows that 120 observations are certainly not natural.

0    123392
1       120
Name: FilterSymbols, dtype: int64

In [56]:
# Third filter: Location / Region based company entity identifier
data["dmcountry"].value_counts()

def company_legal_entity_identifier(entities):
    for i in entities:
        data[i] = np.where(data["dmfullname"].str.contains(i), 1, 0)

# Main regions considered: Italy, UK, Germany, France, Austria, USA, Netherlands, Sweden, 
#                          Hungary, Luxembourg, Norway, Finland, Ireland and Greece.
regions_entity = ["S.p.a.", "S.r.l.", "Ltd", "PLC", "B.V.", "Limited", "GmbH", 
                       "AG", "UG", "e.V.", "SAS", "SARL", "SA", "OG", "KG", "LLC", 
                       "U.A.", "C.V.", "AB", "HB", "KB", "Zrt.", "Kft.", "S.à r.l.", 
                       "S.A.", "SCSp", "LLCs", "Oy", "LTD","L.T.D.","P.C."]

company_legal_entity_identifier(regions_entity)

data["EntitiesCheck"] = data[regions_entity].apply(lambda row: "+" .join(row.values.astype(str)), axis=1)

data["FilterEntities"] = np.where(data["EntitiesCheck"].str.contains("1"), 1, 0)
data = data.drop(["S.p.a.", "S.r.l.", "Ltd", "PLC", "B.V.", "Limited", "GmbH", 
                       "AG", "UG", "e.V.", "SAS", "SARL", "SA", "OG", "KG", "LLC", 
                       "U.A.", "C.V.", "AB", "HB", "KB", "Zrt.", "Kft.", "S.à r.l.", 
                       "S.A.", "SCSp", "LLCs", "Oy", "LTD","L.T.D.","P.C.", "EntitiesCheck"], axis=1)

data["FilterEntities"].value_counts() 

0    121992
1      1520
Name: FilterEntities, dtype: int64

##### Intermediary Results:

__Dummy Variables:__
  1. __FilterTitle__ indicates, at current moment that, 115982 observations have one of the titles investigated, e.g. Sir, while 7530 doesnt. 
  2. __FilterSymbols__ indicates that 120 observations are certainly not natural persons because they have numeric values in dmfullname.
  3. __FilterEntities__ indicates that 1520 observations have company legal entity names in dmfullname and are therefore certaily not natural persons.

In [57]:
# Looking for potential columns that have a good amount of observations filled.
a = list(data["dmage"].value_counts())
sum(a)

83491

In [58]:
aa = list(data["dmbirthdate"].value_counts())
sum(aa)

82100

In [59]:
b = list(data["dmgender"].value_counts())
sum(b)

119046

##### New filter ideas

Columns dmage, dmbirthdate and dmgender are promising columns to reduce the number of observations I have to check manually once all these columns have more than 80000 filled. Based on this I will continue the filtering process of dummz variable natural_person. 

In [60]:
# Filter Age, Birthdate and Gender

# Coming up with a way to represent empty cells under age , birthday and gender. In this case by filling empty cells with the value -1.
data["FilterAge"] = data["dmage"].fillna(value="-1")  
data["FilterBirthDate"] = data["dmbirthdate"].fillna(value="-1")
data["FilterGender"] = data["dmgender"].fillna(value="-1")

In [61]:
data["FilterGender"].value_counts()

M     98577
F     20469
-1     4466
Name: FilterGender, dtype: int64

In [62]:
# Intermediary csv file with only gender (-1) observations. 
# EVERYTHING HAPPENING FROM HERE IN DATA: MANUAL, IS A TEST.
manual = data[data["FilterGender"] == "-1"]
manual.to_csv("manual_gender_check.csv") # Here, I would have to check 4446 observations.

### Star rating logic

Right now we have 6 filters.  Out of them, 2 are very secure, namely "FilterSymbols" & "FilterEntities", because if you have positive values there you are certainly a non-natural person.

We remain with 4 filters, each has a dummy variable, consequently providing to us 2 variables - therefore, there are 8 variables in questions. There are possibly 8! permutations of these variables - very painful to condition.

At the moment, I can either check 4466 observations manualy or try to scope down further.

The scoping will follow a score mechanism based on only 3 filters: Age, BirthDate and Gender because Filtertitle can be the most misleading and laborious due to these facts: a lot observations to cover in the revision as well as to the possibility of existing companies such as "Dr. Schwarz" or who knows what else.

__Score Structure__

Score can range from 0 to 3 stars. 2 and 3 stars observations won't be revised manually. Observations gain stars based on the existence of a cell value for that attribute or not.

Remember that if an observation had no value in its cell in the columns of FilterAge, FIlterBirthDate and FilterGender they were filled up with value -1. Therefore, the following conditions are being used:



- Age != -1 , BirthDate != -1 , Gender != -1 -> __3 stars rating__
- Age == -1 , BirthDate != -1 , Gender != -1 -> __2 stars rating__
- Age != -1 , BirthDate == -1 , Gender != -1 -> __2 stars rating__
- Age != -1 , BirthDate != -1 , Gender == -1 -> __2 stars rating__

##### Cases that were revised manually
- Age == -1 , BirthDate == -1 , Gender != -1 -> __1 stars rating__
- Age == -1 , BirthDate != -1 , Gender == -1 -> __1 stars rating__
- Age != -1 , BirthDate == -1 , Gender == -1 -> __1 stars rating__
- Age == -1 , BirthDate == -1 , Gender != -1 -> __1 stars rating__
- Age == -1 , BirthDate == -1 , Gender == -1 -> __0 stars rating__

In [63]:
# List of conditions
conditions = [
    (manual['FilterAge'] != "-1") & (manual['FilterBirthDate'] != "-1") & (manual['FilterGender'] != "-1"),
    (manual['FilterAge'] == "-1") & (manual['FilterBirthDate'] != "-1") & (manual['FilterGender'] != "-1"),
    (manual['FilterAge'] != "-1") & (manual['FilterBirthDate'] == "-1") & (manual['FilterGender'] != "-1"),
    (manual['FilterAge'] != "-1") & (manual['FilterBirthDate'] != "-1") & (manual['FilterGender'] == "-1"),
    (manual['FilterAge'] == "-1") & (manual['FilterBirthDate'] == "-1") & (manual['FilterGender'] != "-1"),
    (manual['FilterAge'] == "-1") & (manual['FilterBirthDate'] != "-1") & (manual['FilterGender'] == "-1"),
    (manual['FilterAge'] != "-1") & (manual['FilterBirthDate'] == "-1") & (manual['FilterGender'] == "-1"),
    (manual['FilterAge'] == "-1") & (manual['FilterBirthDate'] == "-1") & (manual['FilterGender'] != "-1"),
    (manual['FilterAge'] == "-1") & (manual['FilterBirthDate'] == "-1") & (manual['FilterGender'] == "-1"),
]

# create a list of the values we want to assign for each condition
values = ['3', '2', '2', '2', '1', '1', '1', '1', '0']

# create a new column and use np.select to assign values to it using our lists as arguments
manual['score'] = np.select(conditions, values)

<ipython-input-63-f35af9bd0680>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manual['score'] = np.select(conditions, values)


In [64]:
manual["score"].value_counts()

0    4153
2     301
1      12
Name: score, dtype: int64

__Result Scoring:__ Based on the first score mechanism I can see that out of 4466 observations, 301 are scored with 2 stars and therefore secure.

### Extending Score mechanism

__Logic:__
- Score != 2 & FilterEntity == 1 -> 'Non'
- Score != 2 & FilterSymbols == 1 -> 'Non'
- Score != 2 & FilterSymbols != 1 -> 'Non'
- Score != 2 & FilterSymbols != 1 -> 'Non'

In [65]:
ext_cond = [(manual["FilterEntities"] == 1) & (manual["score"] != 2),
            (manual["FilterSymbols"] == 1) & (manual["score"] != 2)
            
            
]

ext_values = [
    "non", "non"
]

manual["ExtendedScore"] = np.select(ext_cond, ext_values)

<ipython-input-65-3f05f15de768>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manual["ExtendedScore"] = np.select(ext_cond, ext_values)


In [66]:
manual["ExtendedScore"].value_counts()

0      3683
non     783
Name: ExtendedScore, dtype: int64

In [67]:
### Final extension to score mechanism
final_conditions = [(manual["ExtendedScore"] != "non") & (manual["FilterTitle"] == 1),
                    
]

final_values = [
    "natural",
]

manual["ExtendedFinalScore"] = np.select(final_conditions, final_values)

<ipython-input-67-482ac4cd94f3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manual["ExtendedFinalScore"] = np.select(final_conditions, final_values)


In [68]:
manual["ExtendedFinalScore"].value_counts()

0          4380
natural      86
Name: ExtendedFinalScore, dtype: int64

## Final thoughts on the preprocessing before I engage in manual work.

### Remember that I am operating within the 4466 observations that are not certain.

__Overview results:__

- Encoding dmfullname based on natural person titles: 115982 observations have titles / 7530 observations have no titles.
- Enconding dmfullname based on numeric symbols: 123392 observations have no numbers / 120 observations have numbers in dmfullname.
- Encoding dmfullname based on entitity legal titles: 121992 observations have no entity titles / 1520 observations have entity titles in dmfullname.

- Encoding dmAge, dmBirthDate & dmGender by checking if a value exists or not: 
    - dmGender -> Males: 98577 and Females: 20469 which together (a.k.a Natural) = 119046; -1 (a.k.a Non-natural and possible natural) = 4466
    - dmAge -> Age given: 83491 , Age not given: Total Observations - 83491
    - dmBirthDay -> BirthDay given: 82100, BirthDay not given: Total Observations - 82100

__Upon Ecoding Strategies, summary of Scoring mechanism:__

- 3 scores were created
    - Column 'score' is a variable dependent on a given or not value encoding based on dmAge, dmGender and dmBirthDate; -> 301 observations less to manually check;
    - Column 'ExtendedScore' is a variable dependent on a the scoring mechanism of column 'Score' and filters 'FilterSymbols' and 'FilterEntities'. -> 783 observations less to manually check;
    - Column 'ExtendedFinalScore' is a variable dependent on scoring mechanism 'ExtendedScore' and column FilterTitle. -> 86 observations less to manually check.
    
### Final steps:

In [69]:
score_conditions = [
    (manual['ExtendedFinalScore'] == 'natural'),
    (manual['ExtendedFinalScore'] != 'natural') & (manual['ExtendedScore'] == 'non'),
    (manual['ExtendedFinalScore'] != 'natural') & (manual['ExtendedScore'] != 'non')  
]

score_value = ["natural",
               "non-natural",
               "revision"          
]

manual["NaturalPerson"] = np.select(score_conditions, score_value)

<ipython-input-69-5c357a6ccbff>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manual["NaturalPerson"] = np.select(score_conditions, score_value)


In [70]:
manual["NaturalPerson"].value_counts()
manual.to_csv("revision.csv") # Here, I would have to check 3597 observations.

#### I need to revise 3597 instead of 4466 to define dummy variables NaturalPerson and GenderManual officialy.

#### Final steps for task:

1. Embeded checked manual data with secure data.
2. Finish 'NaturalPerson' encoding
3. Create GenderManual dummy

### Merging logic: 

I have merged both datas horizontally based on their index number which matched. This implies the following: data frame "manual_checked" had columns and values which were not part of data frame "data". The last operation merged the data and gave to the observations which were not consider in data frane "manual_checked" a NaN value. All the observations with NaN value are actual naturalPersons because they presented a gender attribute. These are the 119046 observations mentioned before.

In [71]:
checked_data = pd.read_table("revision_manual.csv", sep=",", index_col=0)
checked_data["NaturalPerson"].value_counts()

natural        2290
non-natural    2176
Name: NaturalPerson, dtype: int64

In [72]:
data = data.astype(str)
checked_data = checked_data.astype(str)

final = data.combine_first(checked_data) # CHECKING
final["NaturalPerson"].value_counts() # Merge numbers match numbers from "checked_data"
#final

natural        2290
non-natural    2176
Name: NaturalPerson, dtype: int64

In [73]:
final["NaturalPerson"] = final["NaturalPerson"].fillna("natural") # Filling, NaN observations with Natural because all of them had Gender.
final["NaturalPerson"].value_counts() # Done. 

natural        121336
non-natural      2176
Name: NaturalPerson, dtype: int64

#### Task 1: dummy variable "NaturalPerson" is done. 

__Result:__ 121336 observations are natural persons while 2176 aren't.

### Task 2: dummy variable: gender_manual

__Logic:__ 
- 119046 observations have gender given. 
- 4466 are insecure observations, from which after manual checking them I can say, 2176 are non-natural persons.
- 2290 should be checked / reviewed manually -> Here, I could check the data manually but there names I am not familiar with. For this reason, I have choosen to apply an algorithm to the column "dmfirstname".

In [74]:
## TO DO: Separate the 2290 observation that are natural persons and apply algo in them to define their gender.
genderize = checked_data[checked_data["NaturalPerson"] == "natural"]
genderize # 2290 observations that are subjects of gender classification are ready!

,isin,v1,companynamelatinalphabet,bvdidnumber,name,cname,dmfullname,dmuciuniquecontactidentifie,dmjobtitleinenglish,dmjobtitle,...,FilterTitle,FilterSymbols,FilterEntities,FilterAge,FilterBirthDate,FilterGender,score,ExtendedScore,ExtendedFinalScore,NaturalPerson
190,FR0000060618,60,RALLYE,FR054500574,RALLYE,RALLYE,Fonciere Euris,C003873150,Director (Board of Directors),Director (Board of Directors),...,0,0,0,-1,-1,-1,0,0,0,natural
297,GB00B39QB067,4037,THE RETHINK GROUP LIMITED,GB05078352,RETHINK GROUP (THE) DEAD - 09/12/14,RETHINK GROUP PLC.,ST JAMES DIRECTORS LIMITED,C007514081,Director (occupation: Limited Company),Director (occupation: Limited Company),...,0,0,0,-1,-1,-1,0,0,0,natural
560,BE0974280126,9221,BONE THERAPEUTICS SA,BE0882015654,BONE THERAPEUTIC,BONE THERAPEUTIC,Enrico Bastianelli,C058959216,Director,Administrateur,...,0,0,0,-1,-1,-1,0,0,0,natural
655,BE0003766806,2712,ION BEAM APPLICATIONS SA,BE0428750985,ION BEAM APPLICATIONS,ION BEAM APPLICATIONS SA,CONSULTANCE MARCEL MILLER,C112065648,Director,Administrateur,...,0,0,0,-1,-1,-1,0,0,0,natural
785,BE0003678894,3322,BEFIMMO SA,BE0455835167,BEFIMMO,BEFIMMO SCA,KADEES,C178679878,Director,Administrateur,...,0,0,0,-1,-1,-1,0,0,0,natural
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123505,MT0000150103,8963,LOQUS HOLDINGS PLC,MTC27140,LOQUS HOLDINGS,LOQUS HOLDINGS PLC.,Joannie Grima,P649250964,Director,Director,...,0,0,0,-1,-1,-1,0,0,0,natural
123506,GRS354003006,7456,KTIMA KOSTAS LAZARIDIS S.A,GR051520819000,KTIMA KOSTAS LAZARIDIS,KTIMA KOSTAS LAZARIDIS,Julia Lazaridis,P649252635,Chairman,Chairman,...,0,0,0,-1,-1,-1,0,0,0,natural
123507,GRS354003006,7456,KTIMA KOSTAS LAZARIDIS S.A,GR051520819000,KTIMA KOSTAS LAZARIDIS,KTIMA KOSTAS LAZARIDIS,Triantaphyllos Tsamantanis,P649252636,Non-Executive-Independent,Non-Executive-Independent,...,0,0,0,-1,-1,-1,0,0,0,natural
123508,CH0496451508,359,SOFTWAREONE HOLDING AG,CHCHE384378612,SOFTWAREONE HOLDING,SOFTWAREONE HOLDING AG,Daniel Von Stockar,P649256001,Chairman Supervisory Board,Chairman Supervisory Board,...,0,0,0,-1,-1,-1,0,0,0,natural


In [75]:
genderize["dmfirstname"].value_counts().sum() # Out of those 2290, 2144 have a dmfirstname given. Which implies that 146 observations would still have to be checked manually.
genderize["dmfirstname"] = genderize["dmfirstname"].fillna(value="revision") # Marking all manual checks with 0

<ipython-input-75-0dc7de8f9e69>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genderize["dmfirstname"] = genderize["dmfirstname"].fillna(value="revision") # Marking all manual checks with 0


In [76]:
genderize["dmfirstname"].value_counts() # Checking progress

nan           146
Michael        16
David          16
John           13
Andrea         12
             ... 
Golba           1
Lm              1
Vassileios      1
Elsa            1
Saurin          1
Name: dmfirstname, Length: 1499, dtype: int64

In [77]:
gender_revision_manual = genderize[genderize["dmfirstname"] == "revision"]
gender_revision_manual.to_csv("gender_revision_manual.csv") # Going manual

# Merging rquirements
gender_checked = pd.read_table("gender_revision_manual 2.csv", sep=",", index_col=0)
gender_checked = gender_checked.astype(str)

In [78]:
# Merge
intermediary_obj = gender_checked.combine_first(genderize) # Updated and manually check no dmfirstname observations that are natural persons
returning_intermediary_to_final = intermediary_obj.combine_first(final) # Adding back some mistakes regarding "naturalpersons". Previously 2176 non-natural, now 2195.

df_apply_classifier = returning_intermediary_to_final[returning_intermediary_to_final["NaturalPerson"] != "non-natural"]
df_apply_classifier["dmfirstname"].value_counts().sum() # Verification, that all my NaturalPerson have actually a dmfirstname to apply the gender classifier.

df_apply_classifier_predict = df_apply_classifier[df_apply_classifier["dmgender"] == "nan"]
df_apply_classifier_names = df_apply_classifier_predict["dmfirstname"]
df_apply_classifier_names.value_counts().sum() # Checking the amount observations I have to predict and numbers matches.

2271

In [79]:
# Credits for the inspiration: https://github.com/Jcharis/Python-Machine-Learning/tree/master/Gender%20Classification%20With%20%20Machine%20Learning

# Setup of classifier
names_data = pd.read_csv('names_dataset.csv') # Total of: 181800 female names & 103275 male names
names_data.sex.replace({'F':0,'M':1},inplace=True) # Female is 0, Male is 1

Xfeatures =names_data['name']

# Feature Extraction 
cv = CountVectorizer()
X = cv.fit_transform(Xfeatures)

In [80]:
### Train Part
from sklearn.model_selection import train_test_split

# Labels
y = names_data.sex
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.6398163206734908

In [81]:
### Test / Prediction Part: Female is 0, Male is 1

def genderpredictor(a):
    test_name = [a]
    vector = cv.transform(test_name).toarray()
    if clf.predict(vector) == 0:
        print("Female")
    else:
        print("Male")
        
# Gender list that I want to predict in my model
cake = ["Mark", "Carla"]

bayes_classifier = {}

for i in cake:
    bayes_classifier[i] = genderpredictor(i)

Male
Female


In [82]:
df_apply_classifier_names.value_counts() # There are 1557 different first names. 
df_apply_classifier_names.value_counts().sum() # Their ,sum() add up to 2271\
df_apply_classifier_names

190             Fonciere
560               Enrico
888            Gianbeppi
990                 Marc
1220              Thomas
               ...      
123505           Joannie
123506             Julia
123507    Triantaphyllos
123508            Daniel
123510         Anastasia
Name: dmfirstname, Length: 2271, dtype: object

In [83]:
# By Analogy most female names ends in 'A' or 'E' or has the sound of 'A'
def features(name):
    name = name.lower()
    return {
        'first-letter': name[0], # First letter
        'first2-letters': name[0:2], # First 2 letters
        'first3-letters': name[0:3], # First 3 letters
        'last-letter': name[-1],
        'last2-letters': name[-2:],
        'last3-letters': name[-3:],
    }

# Vectorize the features function
features = np.vectorize(features)

# Extract the features for the dataset
df_X = features(names_data['name'])
df_y = names_data['sex']

from sklearn.feature_extraction import DictVectorizer
 
corpus = features(["Mike", "Julia"])
dv = DictVectorizer()
dv.fit(corpus)
transformed = dv.transform(corpus)
dfX_train, dfX_test, dfy_train, dfy_test = train_test_split(df_X, df_y, test_size=0.33, random_state=42)
dv = DictVectorizer()
dv.fit_transform(dfX_train)

<63666x8194 sparse matrix of type '<class 'numpy.float64'>'
	with 381996 stored elements in Compressed Sparse Row format>

In [84]:
# Model building Using DecisionTree instead of NaiveBayes

from sklearn.tree import DecisionTreeClassifier
 
dclf = DecisionTreeClassifier()
my_xfeatures =dv.transform(dfX_train)
dclf.fit(my_xfeatures, dfy_train)

DecisionTreeClassifier()

In [85]:
# Predicting Gender of Name
# Male is 1,female = 0

# A function to do it
def genderpredictor1(a):
    test_name1 = [a]
    transform_dv =dv.transform(features(test_name1))
    vector = transform_dv.toarray()
    if dclf.predict(vector) == 0:
        print("Female")
    else:
        print("Male")
        
namelist = list(df_apply_classifier_names)
classified = {}

for n in namelist:
    classified[n] = genderpredictor1(n) # Ctrl + c to copy classification and adding manually to a .csv file.

Female
Male
Male
Male
Male
Female
Male
Male
Male
Male
Male
Male
Male
Male
Male
Female
Female
Female
Male
Male
Male
Male
Male
Male
Male
Male
Female
Female
Male
Female
Female
Female
Male
Male
Male
Male
Male
Female
Male
Female
Female
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Female
Female
Male
Male
Male
Male
Male
Female
Male
Male
Male
Male
Male
Male
Male
Male
Female
Male
Male
Female
Male
Female
Male
Female
Female
Male
Male
Male
Female
Male
Male
Male
Female
Male
Female
Male
Female
Female
Male
Female
Male
Female
Male
Male
Male
Male
Female
Male
Male
Female
Female
Female
Male
Female
Female
Male
Male
Male
Male
Male
Male
Female
Female
Male
Male
Female
Male
Male
Female
Female
Female
Female
Female
Female
Female
Female
Male
Female
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Female
Female
Male
Male
Male
Female
Male
Male
Male
Female
Male
Male
Female
Male
Male
Male
Male
Male
Male
Female
Male
Female
Male
Male
Male
Female
Female
Female
Female
Male
Male
Male
Male
Female

Female
Male
Female
Male
Male
Female
Female
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Female
Female
Male
Male
Female
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Female
Male
Female
Male
Female
Female
Male
Male
Male
Male
Male
Female
Male
Female
Male
Female
Male
Female
Male
Female
Male
Female
Female
Male
Male
Male
Female
Female
Female
Male
Male
Female
Female
Male
Male
Female
Male
Female
Male
Female
Male
Female
Male
Male
Female
Male
Male
Male
Male
Male
Male
Female
Male
Male
Male
Male
Female
Female
Male
Female
Male
Male
Female
Female
Male
Male
Male
Male
Male
Male
Male
Female
Male
Male
Male
Female
Male
Male
Male
Male
Female
Male
Male
Male
Male
Female
Male
Male
Male
Male
Female
Male
Female
Male
Male
Female
Male
Male
Male
Female
Male
Male
Female
Male
Male
Male
Female
Female
Male
Male
Female
Female
Male
Female
Male
Male
Male
Male
Male
Male
Male
Male
Female
Male
Male
Male
Male
Male
Male
Male
Male
Male
Male
Female
Male
Male
Male
Male
Male
Male
Male
Male
Femal

In [86]:
## Accuracy of Models Decision Tree Classifier Works better than Naive Bayes

# Accuracy on training set
print(dclf.score(dv.transform(dfX_train), dfy_train))

# Accuracy on test set
print(dclf.score(dv.transform(dfX_test), dfy_test))

0.9888951716771903
0.8673745974042539


In [36]:
df_apply_classifier_predict.to_csv("df_apply_classifier_predict.csv")
df_apply_classifier_predict["dmgender"].value_counts() # Ctrl + C classification should be copied to this location.

nan    2271
Name: dmgender, dtype: int64

In [37]:
predicted = pd.read_table("one.csv", sep=",", index_col=0)
predicted["dmgender"].value_counts() # Result from the observations that were NaturalPersons, after manual check, and didt have firstname nor gender given

M    1448
F     823
Name: dmgender, dtype: int64

In [38]:
returning_intermediary_to_final["dmgender"] = returning_intermediary_to_final["dmgender"].replace("","NaN")
combiner = predicted.combine_first(returning_intermediary_to_final)
combiner

,ExtendedFinalScore,ExtendedScore,FilterAge,FilterBirthDate,FilterEntities,FilterGender,FilterSymbols,FilterTitle,NaturalPerson,bvdidnumber,...,dmsalutation,dmsuffix,dmtitle,dmtypeofrole,dmuciuniquecontactidentifie,gender,isin,name,score,v1
1,0,non,-1,-1,1,-1,0,0,non-natural,NL08142836,...,nan,nan,nan,ExeB,C000459947,nan,NL0012015705,JUST EAT TAKEAWA,0,2225
2,0,non,-1,-1,1,-1,0,0,non-natural,NL24004888,...,nan,nan,nan,BoD,C001024491,nan,NL0000383800,SMIT INTL.CERTS. DEAD - 05/05/10,0,1837
3,0,non,-1,-1,1,-1,0,0,non-natural,NL24041906,...,nan,nan,nan,ExeB,C001055291,nan,NL0000289783,ROBECO SUST.GLB. STARS EQTIES.FD.,0,12368
4,0,non,-1,-1,1,-1,0,0,non-natural,NL27234422,...,nan,nan,nan,BoD,C001133291,nan,NL0000350361,EXACT HOLDING DEAD - 31/03/15,0,3289
5,0,non,-1,-1,1,-1,0,0,non-natural,NL30277334,...,nan,nan,nan,ExeB,C001200105,nan,NL0009508720,LBI INTERNATIONAL DEAD - 07/03/13,0,13696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123508,0,0,-1,-1,0,-1,0,0,natural,CHCHE384378612,...,nan,nan,nan,SupB,P649256001,nan,CH0496451508,SOFTWAREONE HOLDING,0,359
123509,NaN,NaN,-1,-1,0,F,0,1,natural,CYC132218,...,Ms,nan,nan,BoD,P649258636,F,CY0009731015,XXI CENTURY INVS.PUBLIC DEAD - 03/12/13,NaN,8461
123510,0,0,-1,-1,0,-1,0,0,natural,GR094000952,...,nan,nan,nan,SenMan,P649259308,nan,GRS228003000,S&B INDUSTRIAL MRLS. DEAD - 12/07/13,0,4116
123511,NaN,NaN,-1,-1,0,M,0,1,natural,CYC12013,...,Mr,nan,nan,BoD,P649264673,M,CY0007400613,CHRIS IOANNOU,NaN,8843


In [39]:
#returning_intermediary_to_final["NaturalPerson"].value_counts()
combiner["NaturalPerson"].value_counts() # NaturalPersons checked

natural        121317
non-natural      2195
Name: NaturalPerson, dtype: int64

In [40]:
#returning_intermediary_to_final["dmgender"].value_counts()
combiner["dmgender"].value_counts() # Gender checked checked

M      100025
F       21292
nan      2195
Name: dmgender, dtype: int64

In [41]:
### ENCODING OBSERVATIONS
categories = ["M", "F"] # Males == 1; Females == 0;
cleanup = [1, 0]

persons = ["natural", "non-natural"] # Natural == 1; Non-natural == 0;

combiner["GenderManual"] = combiner["dmgender"].replace(categories, cleanup)
combiner["NaturalPerson"] = combiner["NaturalPerson"].replace(persons, cleanup)
#combiner["NaturalPerson"].value_counts()
#combiner["GenderManual"].value_counts()
combiner.head()

,ExtendedFinalScore,ExtendedScore,FilterAge,FilterBirthDate,FilterEntities,FilterGender,FilterSymbols,FilterTitle,NaturalPerson,bvdidnumber,...,dmsuffix,dmtitle,dmtypeofrole,dmuciuniquecontactidentifie,gender,isin,name,score,v1,GenderManual
1,0,non,-1,-1,1,-1,0,0,0,NL08142836,...,nan,nan,ExeB,C000459947,nan,NL0012015705,JUST EAT TAKEAWA,0,2225,nan
2,0,non,-1,-1,1,-1,0,0,0,NL24004888,...,nan,nan,BoD,C001024491,nan,NL0000383800,SMIT INTL.CERTS. DEAD - 05/05/10,0,1837,nan
3,0,non,-1,-1,1,-1,0,0,0,NL24041906,...,nan,nan,ExeB,C001055291,nan,NL0000289783,ROBECO SUST.GLB. STARS EQTIES.FD.,0,12368,nan
4,0,non,-1,-1,1,-1,0,0,0,NL27234422,...,nan,nan,BoD,C001133291,nan,NL0000350361,EXACT HOLDING DEAD - 31/03/15,0,3289,nan
5,0,non,-1,-1,1,-1,0,0,0,NL30277334,...,nan,nan,ExeB,C001200105,nan,NL0009508720,LBI INTERNATIONAL DEAD - 07/03/13,0,13696,nan


In [42]:
### CLEANind & .CSV 
drops = [ 'ExtendedFinalScore',
 'ExtendedScore',
 'score',
 'FilterAge',
 'FilterBirthDate',
 'FilterEntities',
 'FilterGender',
 'FilterSymbols',
 'FilterTitle']

dataFinal = combiner.drop(drops, axis=1)

column_names = ['isin',
 'v1',
 'companynamelatinalphabet',
 'bvdidnumber',
 'name',
 'cname',
 'dmfullname',
 'dmuciuniquecontactidentifie',
 'dmjobtitleinenglish',
 'dmjobtitle',
 'dmappointmentdate',
 'dmresignationdate',
 'dmtitle',
 'dmsalutation',
 'dmfirstname',
 'dmmiddlename',
 'dmlastname',
 'dmsuffix',
 'dmgender',
 'dmbirthdate',
 'dmage',
 'dmagebracket',
 'dmcountryiesofnationality',
 'dmaddress',
 'dmcountry',
 'dmemailaddress',
 'dmbiography',
 'dmtypeofrole',
 'dmboardcommitteeordepartmen',
 'dmlevelofresponsibility',
 'dmalsoashareholder',
 'dmconfirmationdates',
 'dmdateslastreceivedfromip',
 'dmnotvalidafterdate',
 'dminformationsources',
 'dminformationproviders',
 'dmcollege',
 'dmdegreecode',
 'dmmajor',
 'dmgraduationdate',
 'dmcorp',
 'gender',
 'dmcorrespondingbvdidwhenapp',
 'dmasanycategory',
 'dmcurrentorprevious',
 'dmbirthplace',
 'dmhasasignatoryright',
 'dmhasapowerofattorney',
 'dmnoofcosinwhichacurrent',
 'compensationsalaryeur', 'NaturalPerson', 'GenderManual']

dataFinal = dataFinal.reindex(columns=column_names)
dataFinal.head()

,isin,v1,companynamelatinalphabet,bvdidnumber,name,cname,dmfullname,dmuciuniquecontactidentifie,dmjobtitleinenglish,dmjobtitle,...,dmcorrespondingbvdidwhenapp,dmasanycategory,dmcurrentorprevious,dmbirthplace,dmhasasignatoryright,dmhasapowerofattorney,dmnoofcosinwhichacurrent,compensationsalaryeur,NaturalPerson,GenderManual
1,NL0012015705,2225,TAKEAWAY.COM N.V.,NL08142836,JUST EAT TAKEAWA,JUST EAT TAKEAWAY COM NV,GRIBHOLD B.V.,C000459947,Director,Bestuurder,...,NL06089183,No,Previous,nan,No,No,0,nan,0,nan
2,NL0000383800,1837,SMIT INTERNATIONALE NV,NL24004888,SMIT INTL.CERTS. DEAD - 05/05/10,SMIT INTL.,Boskalis Holding B.V.,C001024491,Directeur,Directeur,...,NL23056607,Yes,Current,nan,No,No,3,nan,0,nan
3,NL0000289783,12368,ROBECO GLOBAL STARS EQUITIES FUND N.V.,NL24041906,ROBECO SUST.GLB. STARS EQTIES.FD.,ROBECO GLB.STARS EQ.FD.,Robeco Fund Management B.V.,C001055291,Director,Bestuurder,...,nan,nan,nan,nan,nan,nan,nan,nan,0,nan
4,NL0000350361,3289,EXACT HOLDING NV,NL27234422,EXACT HOLDING DEAD - 31/03/15,EXACT HOLDING NV,Exact Management B.V.,C001133291,Directeur,Directeur,...,NL27228442,No,Current,nan,No,No,nan,nan,0,nan
5,NL0009508720,13696,LBI INTERNATIONAL N.V.,NL30277334,LBI INTERNATIONAL DEAD - 07/03/13,LBI INTERNATIONAL NV,Fint Management B.V.,C001200105,Director,Bestuurder,...,nan,nan,nan,nan,nan,nan,nan,nan,0,nan


In [43]:
dataFinal.to_csv("Lucas_final.csv")

### Revision

I realized I have worked only with the set of observations which were uncertain under the column "dmgender". The issue was that after working with this set I have not, doubled check the other set of column "dmgender" - indirectly assuming that the other set was reliable even thought we can't be sure of that.

So what have to be done is: 


    Apply the same procedures to the other set of dmgender. 

In [44]:
dataFinal["NaturalPerson"].value_counts()

1    121317
0      2195
Name: NaturalPerson, dtype: int64

In [46]:
# Preparing set of natural persons to Score Mechanism - reduced version of 3starFilter to 2starFilter because Gender is given by all observations now.

setNatural = dataFinal[dataFinal["NaturalPerson"] == 1]

setNatural = setNatural.astype("object")

setNatural["FilterAge"] = setNatural["dmage"] 
setNatural["FilterBirthDate"] = setNatural["dmbirthdate"]

encode_nan = {
    "FilterAge": {"nan": -1},
    "FilterBirthDate": {"nan": -1}
}

setNatural.replace(encode_nan, inplace=True)

#setNatural["FilterBirthDate"].value_counts()
#setNatural["FilterDate"].value_counts()

# List of conditions
conditions_set_natural = [
    (setNatural['FilterAge'] != -1) & (setNatural['FilterBirthDate'] != -1),
    (setNatural['FilterAge'] == -1) & (setNatural['FilterBirthDate'] != -1),
    (setNatural['FilterAge'] != -1) & (setNatural['FilterBirthDate'] == -1),
    (setNatural['FilterAge'] == -1) & (setNatural['FilterBirthDate'] == -1),
]

# create a list of the values we want to assign for each condition
values_set_natural = ['2', '1', '1', '0']

# create a new column and use np.select to assign values to it using our lists as arguments
setNatural['Score'] = np.select(conditions_set_natural, values_set_natural)
#setNatural = setNatural.drop("FilterGender", axis=1)

setNatural['Score'].value_counts()

2    82087
0    37821
1     1409
Name: Score, dtype: int64

### In between Interpretation 

So out of the "natural persons" set, there are 82087 with a 2 star rating (__adjust rating__, if I had consider Gender I would have received 3 star ratings in all of these) and there are 1409 observations with a 1 star rating (__adjust rating__, if I had consider Gender I would have received 2 star ratings in all of these). So there are 37821 observations that I could verify manually or scope down. 

I will try to scope them down using 2 new filtes: FilterTitle and FilterEntities.

In [47]:
setNaturalrevision = setNatural[setNatural['Score'] == "0"]

def special_person_address_identifier(address):
    for i in address:
        setNaturalrevision[i] = np.where(setNaturalrevision["dmfullname"].str.contains(i), 1, 0)
    
titles = ["Sir", "Madam", "Ms", "Mr", "Mrs","Miss", "Dr", "Professor"]
special_person_address_identifier(titles)


setNaturalrevision["TitleCheck"] = setNaturalrevision[titles].apply(lambda row: "+" .join(row.values.astype(str)), axis=1)

setNaturalrevision["FilterTitle"] = np.where(setNaturalrevision["TitleCheck"].str.contains("1"), 1, 0)
setNaturalrevision = setNaturalrevision.drop(["Sir", "Madam", "Ms", "Mr", "Mrs","Miss", "Dr", "Professor","TitleCheck"], axis=1)

def company_legal_entity_identifier(entities):
    for i in entities:
        setNaturalrevision[i] = np.where(setNaturalrevision["dmfullname"].str.contains(i), 1, 0)

# Main regions considered: Italy, UK, Germany, France, Austria, USA, Netherlands, Sweden, 
#                          Hungary, Luxembourg, Norway, Finland, Ireland and Greece.
regions_entity = ["S.p.a.", "S.r.l.", "Ltd", "PLC", "B.V.", "Limited", "GmbH", 
                       "AG", "UG", "e.V.", "SAS", "SARL", "SA", "OG", "KG", "LLC", 
                       "U.A.", "C.V.", "AB", "HB", "KB", "Zrt.", "Kft.", "S.à r.l.", 
                       "S.A.", "SCSp", "LLCs", "Oy", "LTD","L.T.D.","P.C."]

company_legal_entity_identifier(regions_entity)

setNaturalrevision["EntitiesCheck"] = setNaturalrevision[regions_entity].apply(lambda row: "+" .join(row.values.astype(str)), axis=1)

setNaturalrevision["FilterEntities"] = np.where(setNaturalrevision["EntitiesCheck"].str.contains("1"), 1, 0)
setNaturalrevision = setNaturalrevision.drop(["S.p.a.", "S.r.l.", "Ltd", "PLC", "B.V.", "Limited", "GmbH", 
                       "AG", "UG", "e.V.", "SAS", "SARL", "SA", "OG", "KG", "LLC", 
                       "U.A.", "C.V.", "AB", "HB", "KB", "Zrt.", "Kft.", "S.à r.l.", 
                       "S.A.", "SCSp", "LLCs", "Oy", "LTD","L.T.D.","P.C.", "EntitiesCheck"], axis=1)

setNaturalrevision.head()


<ipython-input-47-e489f78e2900>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setNaturalrevision[i] = np.where(setNaturalrevision["dmfullname"].str.contains(i), 1, 0)
<ipython-input-47-e489f78e2900>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setNaturalrevision["TitleCheck"] = setNaturalrevision[titles].apply(lambda row: "+" .join(row.values.astype(str)), axis=1)
<ipython-input-47-e489f78e2900>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,isin,v1,companynamelatinalphabet,bvdidnumber,name,cname,dmfullname,dmuciuniquecontactidentifie,dmjobtitleinenglish,dmjobtitle,...,dmhasapowerofattorney,dmnoofcosinwhichacurrent,compensationsalaryeur,NaturalPerson,GenderManual,FilterAge,FilterBirthDate,Score,FilterTitle,FilterEntities
190,FR0000060618,60,RALLYE,FR054500574,RALLYE,RALLYE,Fonciere Euris,C003873150,Director (Board of Directors),Director (Board of Directors),...,No,nan,nan,1,0,-1,-1,0,0,0
560,BE0974280126,9221,BONE THERAPEUTICS SA,BE0882015654,BONE THERAPEUTIC,BONE THERAPEUTIC,Enrico Bastianelli,C058959216,Director,Administrateur,...,No,3,nan,1,1,-1,-1,0,0,0
888,IT0004370463,4600,RETELIT SPA,IT12897160151,RETI TELEMATICHE ITALIAN,RETI TELEMATICHE ITALIAN,Gianbeppi Fortis,C367722454,Independent Director (Board of Directors),Independent Director (Board of Directors),...,No,2,nan,1,1,-1,-1,0,0,0
990,BE0003586949,11865,BMT,BE0406069021,BMT DEAD - DELIST 17/11/04,BMT,Marc Ooms Sarl,CVA0093698,Vice Chairman of the Board of Directors,Ondervoorzitter Van de Raad Van bestuur,...,nan,nan,nan,1,1,-1,-1,0,0,0
1220,GB0033990283,13112,ARICOM LIMITED,GB04897906,ARICOM DEAD - EX.INTO 257965,ARICOM PLC.,Thomas Sebastian Swithenbank,CVA0321166,Director,Director,...,nan,nan,nan,1,1,-1,-1,0,0,0


In [48]:
# List of conditions
conditions_set_natural_str = [
    (setNaturalrevision['FilterTitle'] == 1) & (setNaturalrevision['FilterEntities'] != 1),
    (setNaturalrevision['FilterTitle'] == 1) & (setNaturalrevision['FilterEntities'] == 1),
    (setNaturalrevision['FilterTitle'] != 1) & (setNaturalrevision['FilterEntities'] == 1),
    (setNaturalrevision['FilterTitle'] != 1) & (setNaturalrevision['FilterEntities'] != 1)
    
]

# create a list of the values we want to assign for each condition
values_set_natural_str = ['1', 'non', 'non','0']

# create a new column and use np.select to assign values to it using our lists as arguments
setNaturalrevision['ScoreBasedString'] = np.select(conditions_set_natural_str, values_set_natural_str)
setNaturalrevision['ScoreBasedString'].value_counts()

1      34987
0       2551
non      283
Name: ScoreBasedString, dtype: int64

### In between Interpretation


So out of the 37821 observations which needed revision, I managed to scope them down to 2551 observations.

1. Review manually 2551 observations
2. Try to understand how this will affect task 02

In [87]:
manual_setNatural = setNaturalrevision[setNaturalrevision['ScoreBasedString'] == "0"]
manual_setNatural.to_csv("manual_setNatural.csv")

In [90]:
manual_setNatural_checked = pd.read_table("manual_setNatural_checked.csv", sep=",", index_col=0)

In [94]:
manual_setNatural_checked["NaturalPerson"].value_counts() # After checking manually I found 2 observation out of the 2551 that are companies. I have changed them directly in the last dataframe.
manual_setNatural_checked[manual_setNatural_checked["NaturalPerson"] == 0]

,isin,v1,companynamelatinalphabet,bvdidnumber,name,cname,dmfullname,dmuciuniquecontactidentifie,dmjobtitleinenglish,dmjobtitle,...,dmnoofcosinwhichacurrent,compensationsalaryeur,NaturalPerson,GenderManual,FilterAge,FilterBirthDate,Score,FilterTitle,FilterEntities,ScoreBasedString
73952,FR0010518985,9024.0,CHAMARRE SA,FR481539526,CHAMARRE DEAD - 23/12/10,CHAMARRE,Distillerie Des Moisans,P130279709,Member of the Supervisory Board,Membre du Conseil de Surveillance,...,0.0,NaN,0,0,-1,-1,0,0,0,0
112698,GRS474003001,6477.0,I. KLOUKINAS - I. LAPPAS S.A.,GR002052601000,KLOUKINAS LAPPAS,KLOUKINAS LAPPAS SA,Nurser Of Athanasios,P452605618,Independent Non Executive Member,Independent Non Executive Member,...,NaN,NaN,0,1,-1,-1,0,0,0,0


In [107]:
## Checking if my changes worked on 

checksdf = pd.read_table("Lucas_final.csv", sep=",", index_col=0)


In [108]:
checksdf["GenderManual"].value_counts()dr

1.0    100023
0.0     21292
Name: GenderManual, dtype: int64